In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

In [16]:
data=pd.read_csv('diabetes.csv') #loading the dataset

In [17]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [18]:
l=['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for i in l:
    data[i]=data[i].replace(0,np.mean(data[i]))

In [19]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.000000,79.799479,33.6,0.627,50,1
1,1,85.0,66.0,29.000000,79.799479,26.6,0.351,31,0
2,8,183.0,64.0,20.536458,79.799479,23.3,0.672,32,1
3,1,89.0,66.0,23.000000,94.000000,28.1,0.167,21,0
4,0,137.0,40.0,35.000000,168.000000,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.000000,180.000000,32.9,0.171,63,0
764,2,122.0,70.0,27.000000,79.799479,36.8,0.340,27,0
765,5,121.0,72.0,23.000000,112.000000,26.2,0.245,30,0
766,1,126.0,60.0,20.536458,79.799479,30.1,0.349,47,1


In [20]:
x=data.iloc[:,:8]
y=data['Outcome']

## Bagging and Pasting

In bagging and pasting , we divide the train data into different samples. Then using only one base estimater or predictor we will do the predictions like hard voting in voting classifier.
Bagging involves the samples picked using replacement ie is records /instances can be simplar also.
Pasting involves the samples picked using without using replacement that is no instance in samples is same.


#### Decision tree classifier is are base estimator.

In [21]:
x_train,x_test,y_train ,y_test= train_test_split(x,y,test_size=.30,random_state=0)

In [22]:
dt=DecisionTreeClassifier(criterion='entropy', random_state=0)
dt.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [23]:
y_pred=dt.predict(x_test)

In [26]:
accuracy_score(y_test,y_pred)

0.7229437229437229

In [27]:
roc_auc_score(y_test,y_pred)

0.674728868996385

### Bagging

In [30]:
bagging=BaggingClassifier(base_estimator=dt,max_samples=100,n_jobs=-1,bootstrap=True)

In [31]:
bagging.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=0,


In [32]:
y_pred1=bagging.predict(x_test)

In [33]:
#checking roc_auc score
from sklearn.metrics import roc_auc_score,accuracy_score

In [34]:
accuracy_score(y_test,y_pred1)

0.7705627705627706

In [35]:
roc_auc_score(y_test,y_pred1) 

0.6990445859872612

Result


dt is giving accuracy of 72% but ensemble method of bagging is giving better accuracy of 75%.

### Pasting

In [50]:
pasting=BaggingClassifier(base_estimator=dt,max_samples=100,n_jobs=-1,bootstrap=False)
#pasting syntax is just same as the bagging with just bootsrap as false  

In [51]:
pasting.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=0,


In [53]:
ypred2=pasting.predict(x_test)

In [56]:
accuracy_score(y_test,ypred2)

0.7012987012987013

In [57]:
roc_auc_score(y_test,ypred2)

0.6016526080220348

Result:
    Bagging is giving better score than pasting

### Out Of Bag Evaluation

Out of bag records are those records which never got picked randomly while picking up the samples in bagging process. Base estimator has never seen those records before.

Out Of Bag Evaluation is done when we take these out of the bag records as validation data because for the base estimator , these records are new. This evaluation will tell us how well the model will predict on test data.
This is done by using hyperparameter oob as true.

In [58]:
bagging1=BaggingClassifier(base_estimator=dt,n_jobs=-1,max_samples=100,oob_score=True)

In [59]:
bagging1.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=0,


In [63]:
bagging1.oob_score_ #this shows how the model will predict on test data

0.7690875232774674

In [66]:
y_ppred=bagging1.predict(x_test)

In [70]:
accuracy_score(y_test,y_ppred) #this is the accuracy of the model on test data and this is close enough to the oob score evaluation

0.7662337662337663